In [1]:
#@title Default title text
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import pandas as pd
import numpy as np

def to_month_id(year, month):
  return (year - 1996) * 12 + month - 6

n_features = 12
lookback = 3
chosen_stocks = ["MCD"] # "AMZN", "MSFT", "IBM", "INTC", "QCOM", "NVDA", "IBM", "ADBE", "EBAY", "CSCO"

In [3]:
dfFeature = pd.read_csv("./lstm_2004_12.csv")
# dfFeature.loc[dfFeature["return"] == 0, "return"] = 1
dfFeature = dfFeature[dfFeature.QAId.isin(chosen_stocks)]
features = dfFeature.drop(columns=['month_id', 'QAId']).as_matrix()
print(features.shape)

(157, 12)


In [4]:
from sklearn.preprocessing import MinMaxScaler
normalizedFeatures = MinMaxScaler().fit_transform(features) \
                                   .reshape(157, len(chosen_stocks), n_features) \
                                   .reshape(157, len(chosen_stocks) * n_features)
print(normalizedFeatures.shape)

(157, 12)


In [5]:
dfTarget = pd.read_csv("./return_2004_40.csv")
dfTarget["return"] = np.sign(dfTarget["return"])
# dfTarget.loc[dfTarget["return"] == 0, "return"] = 1
dfTarget = dfTarget[dfTarget.QAId.isin(chosen_stocks)]
dfTarget = dfTarget[dfTarget.month_id >= (to_month_id(2004, 1) + lookback)]
targets = MinMaxScaler().fit_transform(dfTarget.drop(columns=['month_id', 'QAId']).as_matrix())
y = targets.reshape(157 - lookback + 1, len(chosen_stocks))
print(y.shape)

(155, 1)


In [6]:
from keras.preprocessing.sequence import TimeseriesGenerator

splittedFeature = normalizedFeatures
print(splittedFeature.shape)

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(157, 12)


In [7]:
from keras.preprocessing.sequence import TimeseriesGenerator

x = np.zeros((157 - lookback + 1, lookback, n_features * len(chosen_stocks)))
y_mock = np.zeros((157, len(chosen_stocks)))

i = 0
for train, test in TimeseriesGenerator(splittedFeature, y_mock, length=lookback, batch_size=1):
  if i > 157 - lookback:
    break
  x[i] = train[0]
  i = i + 1

In [8]:
from sklearn.model_selection import PredefinedSplit

test_months = 26
end_point = 157 - lookback + 1
split_point = 157 - lookback + 1 - test_months

x_train = x[0:split_point - 48].reshape(split_point - 48, lookback, n_features * len(chosen_stocks))
y_train = y[0:split_point - 48].reshape(split_point - 48, len(chosen_stocks))
x_validate = x[split_point - 36:split_point].reshape(36, lookback, n_features * len(chosen_stocks))
y_validate = y[split_point - 36:split_point].reshape(36, len(chosen_stocks))
x_test = x[split_point:end_point].reshape(test_months, lookback, n_features * len(chosen_stocks))
y_test = y[split_point:end_point].reshape(test_months, len(chosen_stocks))

print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
print(x_test.shape)
print(y_test.shape)

# ps = PredefinedSplit(np.append(np.negative(np.ones(60 * 465)), np.zeros(24 * 465))).split(x_train)

# for train_ids, test_ids in ps:
#   print(str(train_ids) + ", " + str(test_ids))

(81, 3, 12)
(81, 1)
(36, 3, 12)
(36, 1)
(26, 3, 12)
(26, 1)


In [9]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten, CuDNNLSTM
from keras.regularizers import l1_l2, l2
from keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

np.random.seed(4103)

def create_model(*param):
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(lookback, n_features * len(chosen_stocks)), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(len(chosen_stocks), activation="relu"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, decay=0.0), metrics=['accuracy'], *param)
    return model

In [10]:
batch_size=30

model = create_model()
train_score = model.fit(x_train, y_train, validation_data=(x_validate, y_validate), batch_size=batch_size, epochs=200)
print(train_score)

Train on 81 samples, validate on 36 samples
Epoch 1/200
81/81 [==============================] - 2s 25ms/step - loss: 7.3737 - acc: 0.3580 - val_loss: 2.9211 - val_acc: 0.4444
Epoch 2/200
81/81 [==============================] - 0s 555us/step - loss: 4.6041 - acc: 0.3580 - val_loss: 2.6546 - val_acc: 0.4444
Epoch 3/200
81/81 [==============================] - 0s 515us/step - loss: 3.4750 - acc: 0.3580 - val_loss: 2.5358 - val_acc: 0.4444
Epoch 4/200
81/81 [==============================] - 0s 571us/step - loss: 3.0726 - acc: 0.3580 - val_loss: 2.4556 - val_acc: 0.4444
Epoch 5/200
81/81 [==============================] - 0s 568us/step - loss: 2.9162 - acc: 0.3580 - val_loss: 2.3953 - val_acc: 0.4444
Epoch 6/200
81/81 [==============================] - 0s 528us/step - loss: 2.8504 - acc: 0.3580 - val_loss: 2.3473 - val_acc: 0.4444
Epoch 7/200
81/81 [==============================] - 0s 532us/step - loss: 2.7476 - acc: 0.3580 - val_loss: 2.3082 - val_acc: 0.4444
Epoch 8/200
81/81 [=======

Epoch 62/200
81/81 [==============================] - 0s 487us/step - loss: 1.5559 - acc: 0.3580 - val_loss: 1.3515 - val_acc: 0.4444
Epoch 63/200
81/81 [==============================] - 0s 504us/step - loss: 1.5551 - acc: 0.3580 - val_loss: 1.3399 - val_acc: 0.4444
Epoch 64/200
81/81 [==============================] - 0s 515us/step - loss: 1.5344 - acc: 0.3580 - val_loss: 1.3286 - val_acc: 0.4444
Epoch 65/200
81/81 [==============================] - 0s 513us/step - loss: 1.5137 - acc: 0.3580 - val_loss: 1.3172 - val_acc: 0.4444
Epoch 66/200
81/81 [==============================] - 0s 509us/step - loss: 1.4999 - acc: 0.3580 - val_loss: 1.3060 - val_acc: 0.4444
Epoch 67/200
81/81 [==============================] - 0s 516us/step - loss: 1.4899 - acc: 0.3580 - val_loss: 1.2948 - val_acc: 0.4444
Epoch 68/200
81/81 [==============================] - 0s 508us/step - loss: 1.4736 - acc: 0.3580 - val_loss: 1.2839 - val_acc: 0.4444
Epoch 69/200
81/81 [==============================] - 0s 514us

Epoch 123/200
81/81 [==============================] - 0s 436us/step - loss: 0.8710 - acc: 0.3580 - val_loss: 0.8014 - val_acc: 0.4444
Epoch 124/200
81/81 [==============================] - 0s 479us/step - loss: 0.8509 - acc: 0.3580 - val_loss: 0.7952 - val_acc: 0.4444
Epoch 125/200
81/81 [==============================] - 0s 471us/step - loss: 0.8336 - acc: 0.3580 - val_loss: 0.7891 - val_acc: 0.4444
Epoch 126/200
81/81 [==============================] - 0s 444us/step - loss: 0.8437 - acc: 0.3580 - val_loss: 0.7830 - val_acc: 0.4444
Epoch 127/200
81/81 [==============================] - 0s 445us/step - loss: 0.8294 - acc: 0.3580 - val_loss: 0.7771 - val_acc: 0.4444
Epoch 128/200
81/81 [==============================] - 0s 460us/step - loss: 0.8123 - acc: 0.3580 - val_loss: 0.7714 - val_acc: 0.4444
Epoch 129/200
81/81 [==============================] - 0s 503us/step - loss: 0.8050 - acc: 0.3580 - val_loss: 0.7658 - val_acc: 0.4444
Epoch 130/200
81/81 [==============================] - 

Epoch 184/200
81/81 [==============================] - 0s 470us/step - loss: 0.6644 - acc: 0.5926 - val_loss: 0.6935 - val_acc: 0.5556
Epoch 185/200
81/81 [==============================] - 0s 511us/step - loss: 0.6521 - acc: 0.6296 - val_loss: 0.6935 - val_acc: 0.5556
Epoch 186/200
81/81 [==============================] - 0s 489us/step - loss: 0.6783 - acc: 0.5802 - val_loss: 0.6935 - val_acc: 0.5556
Epoch 187/200
81/81 [==============================] - 0s 482us/step - loss: 0.6693 - acc: 0.5926 - val_loss: 0.6934 - val_acc: 0.5556
Epoch 188/200
81/81 [==============================] - 0s 496us/step - loss: 0.6423 - acc: 0.6420 - val_loss: 0.6935 - val_acc: 0.5556
Epoch 189/200
81/81 [==============================] - 0s 509us/step - loss: 0.6825 - acc: 0.6173 - val_loss: 0.6935 - val_acc: 0.5556
Epoch 190/200
81/81 [==============================] - 0s 514us/step - loss: 0.6745 - acc: 0.6049 - val_loss: 0.6936 - val_acc: 0.5556
Epoch 191/200
81/81 [==============================] - 

In [11]:
import matplotlib.pyplot as plt

test_score = model.evaluate(x_test, y_test, batch_size=batch_size)
for i, mn in enumerate(model.metrics_names):
  print(mn + ": " + str(test_score[i]))

plt.plot(train_score.history["loss"])
plt.plot(train_score.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Metrics")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

26/26 [==============================] - 0s 117us/step
loss: 0.6536162495613098
acc: 0.5769230723381042


In [12]:
from sklearn.metrics import precision_score, recall_score, classification_report

def threshold_tester(result, target, a, b):
  for t in np.arange(a, b + 0.05, 0.05):
    print("thresholder: " + str(t))
    print(classification_report(target, (result >= t).astype(int)))

In [13]:
result = model.predict(x_test, batch_size=batch_size)
print(result)
threshold_tester(result, y_test, 0.1, 0.9)

[[0.5462056 ]
 [0.5277704 ]
 [0.5546787 ]
 [0.581973  ]
 [0.5509689 ]
 [0.5358385 ]
 [0.56629246]
 [0.53196126]
 [0.55738467]
 [0.5831829 ]
 [0.61174273]
 [0.6317868 ]
 [0.581985  ]
 [0.6116964 ]
 [0.62528145]
 [0.6166206 ]
 [0.6030037 ]
 [0.56145865]
 [0.55774766]
 [0.53190035]
 [0.5538004 ]
 [0.56689626]
 [0.60271174]
 [0.5938265 ]
 [0.64058304]
 [0.69727415]]
thresholder: 0.1
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        11
        1.0       0.58      1.00      0.73        15

avg / total       0.33      0.58      0.42        26

thresholder: 0.15000000000000002
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        11
        1.0       0.58      1.00      0.73        15

avg / total       0.33      0.58      0.42        26

thresholder: 0.20000000000000004
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        11
        1.0    

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
import math

buy_list = []
sell_list = []
def second_largest(numbers):
    count = 0
    m1 = m2 = float('-inf')
    for x in numbers:
        count += 1
        if x > m2:
            if x >= m1:
                m1, m2 = x, m1            
            else:
                m2 = x
    return m2 if count >= 2 else None

midpt = (second_largest(map(lambda x: x[0], result)) + min(map(lambda x: x[0], result))) / 2

for i, r in enumerate(result):
  buy_or_sell = 1 if r.item() > midpt * 1.05 else (-1 if r.item() < midpt * 0.95 else 0)
  if r.item() > midpt * 1.05:
    buy_list.append({'month_id': i + 223, 'QAId': chosen_stocks[0]})
  if r.item() < midpt * 0.95:
    sell_list.append({'month_id': i + 223, 'QAId': chosen_stocks[0]})
  print(str(y_test[i].item()) + ": " + str(buy_or_sell))
  if (math.fabs(buy_or_sell - y_test[i].item()) == 2) or (buy_or_sell - y_test[i].item() == 1):
    print("Hey")
print(buy_list)
print(sell_list)

1.0: -1
Hey
0.0: -1
0.0: -1
1.0: 0
0.0: -1
1.0: -1
Hey
0.0: 0
1.0: -1
Hey
1.0: 0
1.0: 0
1.0: 0
1.0: 1
0.0: 0
1.0: 0
1.0: 1
0.0: 1
Hey
0.0: 0
0.0: 0
0.0: 0
0.0: -1
0.0: -1
1.0: 0
1.0: 0
1.0: 0
1.0: 1
1.0: 1
[{'month_id': 234, 'QAId': 'MCD'}, {'month_id': 237, 'QAId': 'MCD'}, {'month_id': 238, 'QAId': 'MCD'}, {'month_id': 247, 'QAId': 'MCD'}, {'month_id': 248, 'QAId': 'MCD'}]
[{'month_id': 223, 'QAId': 'MCD'}, {'month_id': 224, 'QAId': 'MCD'}, {'month_id': 225, 'QAId': 'MCD'}, {'month_id': 227, 'QAId': 'MCD'}, {'month_id': 228, 'QAId': 'MCD'}, {'month_id': 230, 'QAId': 'MCD'}, {'month_id': 242, 'QAId': 'MCD'}, {'month_id': 243, 'QAId': 'MCD'}]


In [15]:
dfBuy = pd.DataFrame(buy_list, columns=["month_id", "QAId"])
dfSell = pd.DataFrame(sell_list, columns=["month_id", "QAId"])
dfBuy.head(5)
dfSell.head(5)

,month_id,QAId
0,223,MCD
1,224,MCD
2,225,MCD
3,227,MCD
4,228,MCD


In [16]:
# from google.colab import files

filename_base = "_".join([stock.lower() for stock in chosen_stocks])
# filename_model = "./" + filename_base + "_model.h5"
# filename_weights = "./" + filename_base + "_weights.h5"
filename_output_buy = "./" + filename_base + "_output_buy.h5"
filename_output_sell = "./" + filename_base + "_output_sell.h5"

# model.save(filename_model)
# model.save_weights(filename_weights)
dfBuy.to_csv(filename_output_buy, index=False)
dfSell.to_csv(filename_output_sell, index=False)

# files.download(filename_model)
# files.download(filename_weights)
# files.download(filename_output_buy)
# files.download(filename_output_sell)